# Config

In [25]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"
import segmentation_models as sm
import tensorflow as tf
from focal_loss import BinaryFocalLoss,sparse_categorical_focal_loss

import os
import cv2
import numpy as np 
import random
import numpy as np
from PIL import Image

from scipy.ndimage.filters import generic_filter

# Input Image Dir
image_dir = "train/images/"
image_target_dir = "train/targets/"

# Output Dir
output_image_dir = "blurred_images/train/images/"
output_target_dir = "blurred_images/train/targets/"

# Noise Parameters 

gaussian_stddev = 0.35
salt_pepper_probability = 0.002

# Majority Filter Parameters 

perform_majority_filtering = 1
neighborhood_size = (3, 3)  # 3x3 neighborhood
threshold = 8  # Threshold value

/var/folders/3y/1js5b25s1b1dk8g7drn7_g8h0000gn/T/ipykernel_16143/4065838880.py:14: DeprecationWarning: Please use `generic_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import generic_filter


## Visualize Noisy Images - ( Optional - For Visualization )

In [44]:
# # List all image files in the directory
# image_files = [f for f in os.listdir(image_dir) if f.endswith((".jpg", ".jpeg", ".png"))]

# indx = random.randint(0, len(image_files))
# image_file = image_files[indx]


# # Read the image
# image_path = os.path.join(image_dir, image_file)
# image = cv2.imread(image_path)
 
# # Add Gaussian noise
# mean = 0
# stddev = gaussian_stddev
# gaussian_noise = np.random.normal(mean, stddev, image.shape).astype(np.uint8)
# noisy_image_gaussian = cv2.add(image, gaussian_noise)

# # Add salt and pepper noise
# salt_pepper_noise = np.zeros(image.shape, np.uint8)
# probability = salt_pepper_probability
# salt = np.where(np.random.rand(*image.shape[:2]) < probability)
# pepper = np.where(np.random.rand(*image.shape[:2]) < probability)
# salt_pepper_noise[salt] = 255
# salt_pepper_noise[pepper] = 0
# noisy_image_salt_pepper = cv2.add(image, salt_pepper_noise)

# cv2.imshow("Original Image", image)
# cv2.imshow("Noisy Image (Salt&Pepper)", noisy_image_salt_pepper)
# cv2.imshow("Noisy Image (Gaussian)", noisy_image_gaussian)

# cv2.waitKey(0)
# cv2.destroyAllWindows()
# cv2.waitKey(1)


## Defining Basic Functions 

In [26]:
def majority_filter(mask, neighborhood_size, threshold):
    def majority_value(arr):
        unique_classes, counts = np.unique(arr, return_counts=True, axis=0)
        max_count = np.max(counts)
        if max_count >= threshold:  # Check if the max count crosses the threshold
            majority_class = unique_classes[np.argmax(counts)]
            return majority_class
        else:
            return arr[int(len(arr)/2)]  # Keep the original value

    filtered_mask = np.zeros_like(mask)

    for channel in range(mask.shape[2]):
        channel_filtered = generic_filter(mask[:, :, channel], majority_value, size=neighborhood_size)
        filtered_mask[:, :, channel] = channel_filtered

    return filtered_mask

def bin_images(image):
    # Convert pixel values less than 2 to 0 and values greater than or equal to 2 to 1
    image[image < 2] = 0
    image[image >= 2] = 1
    
    return image
    
    
def generate_noisy_images(input_dir,image_target_dir,
                          output_image_dir,output_target_dir,
                          gaussian_stddev,salt_pepper_probability,
                         perform_majority_filtering,
                         bin_mask = False,
                         early_stopping = False):

    # Create separate folders for different types of noise
    output_dir_image = os.path.join('', output_image_dir)
    output_dir_mask = os.path.join('', output_target_dir)


    os.makedirs(output_dir_image, exist_ok=True)
    os.makedirs(output_dir_mask, exist_ok=True)


    # List all image files in the directory
    image_files = [f for f in os.listdir(input_dir) if f.endswith((".jpg", ".jpeg", ".png"))]

    for image_file in image_files:
        # Read the image
        image_path = os.path.join(image_dir, image_file)
        image = cv2.imread(image_path)
        
        # Read Mask
        image_mask_path = os.path.join(image_target_dir, image_file[:-4] + '_target.png')
        mask = cv2.imread(image_mask_path)
        #mask = Image.open(image_mask_path)
        if bin_mask:
            mask = bin_images(mask)
        
        if perform_majority_filtering:
            mask = majority_filter(mask, neighborhood_size, threshold)
        
        print(mask.shape)

        # Add Gaussian noise
        mean = 0
        stddev = gaussian_stddev
        gaussian_noise = np.random.normal(mean, stddev, image.shape).astype(np.uint8)
        noisy_image_gaussian = cv2.add(image, gaussian_noise)

        # Add salt and pepper noise
        salt_pepper_noise = np.zeros(image.shape, np.uint8)
        probability = salt_pepper_probability
        salt = np.where(np.random.rand(*image.shape[:2]) < probability)
        pepper = np.where(np.random.rand(*image.shape[:2]) < probability)
        salt_pepper_noise[salt] = 255
        salt_pepper_noise[pepper] = 0
        noisy_image_salt_pepper = cv2.add(image, salt_pepper_noise)



        # Save the noisy images in separate folders
        output_path_gaussian = os.path.join(output_dir_image, image_file[:-4]+'_gaussian'+image_file[-4:])
        output_path_salt_pepper = os.path.join(output_dir_image, image_file[:-4]+'_salt_pepper'+image_file[-4:])
        output_path_original = os.path.join(output_dir_image, image_file)
        
        # Mask for different Noise
        output_path_gaussian_mask = os.path.join(output_dir_mask, image_file[:-4]+'_gaussian_target'+image_file[-4:])
        output_path_salt_pepper_mask = os.path.join(output_dir_mask, image_file[:-4]+'_salt_pepper_target'+image_file[-4:])
        output_path_original_mask = os.path.join(output_dir_mask, image_file[:-4]+'_target'+image_file[-4:])
        
        # Saving
        print(output_path_gaussian)
        cv2.imwrite(output_path_gaussian, noisy_image_gaussian)
        cv2.imwrite(output_path_salt_pepper, noisy_image_salt_pepper)
        cv2.imwrite(output_path_original, image)
        
        print(output_path_gaussian_mask)
        
        cv2.imwrite(output_path_gaussian_mask,mask)
        cv2.imwrite(output_path_salt_pepper_mask,mask)
        cv2.imwrite(output_path_original_mask,mask)
        
#         mask.save(output_path_gaussian_mask)
#         mask.save(output_path_salt_pepper_mask)
#         mask.save(output_path_original_mask)
        
        if early_stopping:
            break

    print("Noisy images saved.")


def load_data(data_dir, resize=False):
    # Set the paths to the directories containing the dataset
    images_dir = os.path.join(data_dir, 'images')
    labels_dir = os.path.join(data_dir, 'targets')

    
    # Set the input image dimensions
    input_shape = (128, 128, 3)  # Adjust as needed

    # Get the list of image filenames
    image_filenames = os.listdir(images_dir)

    # Batch size for loading images
    batch_size = 32

    # Prepare the data for training
    X_train = []
    y_train = []

    # Process images in batches
    for i in range(0, len(image_filenames), batch_size):
        batch_filenames = image_filenames[i:i + batch_size]
        batch_X = []
        batch_y = []

        for filename in batch_filenames:
            # Load pre and post-disaster images
            pre_image_path = os.path.join(images_dir, filename)
            pre_image = cv2.imread(pre_image_path)

            # Load the corresponding label
            label_filename = filename[:-4] + '_target.png'
            label_path = os.path.join(labels_dir, label_filename)
            label = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)

            if resize:
                # Resize the images to the desired shape
                pre_image = cv2.resize(pre_image, (input_shape[0], input_shape[1]))
                label = cv2.resize(label, (input_shape[0], input_shape[1]))

            # Add the data to the batch
            batch_X.append(pre_image)
            batch_y.append(label)

        # Convert the batch lists to numpy arrays
        batch_X = np.array(batch_X)
        print(len(batch_y))

        # Check if all labels in the batch have the same shape
        label_shapes = set([label.shape for label in batch_y])
        if len(label_shapes) > 1:
            raise ValueError("Labels in the batch have different shapes.")

        batch_y = np.array(batch_y)

        # Add the batch data to the training set
        X_train.append(batch_X)
        y_train.append(batch_y)

    # Concatenate the batches to obtain the final training data
    X_train = np.concatenate(X_train)
    y_train = np.concatenate(y_train)

    return X_train, y_train




In [46]:
# Generate Noisy Image Dataset - Skip of Already Created 

generate_noisy_images(image_dir,image_target_dir,
                      output_image_dir,output_target_dir,
                      gaussian_stddev,salt_pepper_probability,
                     perform_majority_filtering = 1 ,
                     bin_mask = True,
                     early_stopping = False)


(1024, 1024, 3)
blurred_images/train/images/hurricane-harvey_00000015_pre_disaster_gaussian.png
blurred_images/train/targets/hurricane-harvey_00000015_pre_disaster_gaussian_target.png
Noisy images saved.


### Testing - No Need to run 

In [47]:
# output_target_dir = "train/targets/"
# output_dir_mask = os.path.join('', output_target_dir)

# # List all image files in the directory
# image_files = [f for f in os.listdir(output_dir_mask) if f.endswith((".jpg", ".jpeg", ".png"))]

# max_pixels = 0
# max_image_path = ''

# for image_file in image_files:
#     # Read the image
#     image_path = os.path.join(output_dir_mask, image_file)
#     image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

#     # Count the pixels with value 2 and above
#     if 2 in np.unique(image):
#         print(image_path)
#         print(np.unique(image))
#         print("-------------------")


# #print("Image with the maximum number of pixels (value >= 2):", max_image_path)



In [48]:
# # output_target_dir = "train/targets/"
# # output_dir_mask = os.path.join('', output_target_dir)

# # # List all image files in the directory
# # image_files = [f for f in os.listdir(output_dir_mask) if f.endswith((".jpg", ".jpeg", ".png"))]

# # max_pixels = 0
# # max_image_path = ''

# # for image_file in image_files:
# #     # Read the image
# #     image_path = os.path.join(output_dir_mask, image_file)
# #     image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# #     # Count the pixels with value 2 and above
# #     count = np.sum(image >= 2)

# #     # Update the maximum pixel count and image path if necessary
# #     if count > max_pixels:
# #         max_pixels = count
# #         max_image_path = image_path

# # print("Image with the maximum number of pixels (value >= 2):", max_image_path)

# ## --- >train/targets/palu-tsunami_00000118_post_disaster_target.png


# # Read the image
# def bin_images(image):
#     # Convert pixel values less than 2 to 0 and values greater than or equal to 2 to 1
#     image[image < 2] = 0
#     image[image >= 2] = 1
    
#     return image

# image = cv2.imread('train/images/palu-tsunami_00000118_post_disaster.png')
# image_mask = cv2.imread('train/targets/palu-tsunami_00000118_post_disaster_target.png',cv2.IMREAD_GRAYSCALE)
# binned_mask = bin_images(image_mask)


# cv2.imshow("Original Image", image)
# cv2.imshow("Original Segment", cv2.bitwise_and(image, image, mask=image_mask))
# cv2.imshow("Binned Segment", cv2.bitwise_and(image, image, mask=binned_mask))

# cv2.waitKey(0)
# cv2.destroyAllWindows()
# cv2.waitKey(1)




## Loading Model Data 

In [49]:
# BACKBONE = 'resnet50'

# (X,y) = load_data('blurred_images/train',resize = False)

# from keras.utils import normalize
# from segmentation_models import get_preprocessing
# from sklearn.model_selection import train_test_split


# y = np.expand_dims(y, axis=3)

# n_classes = 2

# from keras.utils import to_categorical
# train_masks_cat = to_categorical(y, num_classes=n_classes)
# y_train_cat = train_masks_cat.reshape((y.shape[0], y.shape[1], y.shape[2], n_classes))

# X_train, X_val, y_train, y_val  = train_test_split(X, y_train_cat, test_size=0.1, random_state=1)


3


## Training Model 

In [27]:
'''
train 
    train_images 
                train
    train_masks
                train


'''

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the directories for images and masks
train_images_dir = "blurred_images/train/train_images"
train_masks_dir = "blurred_images/train/train_masks"

seed=24
batch_size = 32


# Set up the image data generator for training data
image_datagen = ImageDataGenerator()
mask_datagen = ImageDataGenerator()

# Create the generators for training data
image_generator = image_datagen.flow_from_directory(
    train_images_dir,
    class_mode=None,
    batch_size = batch_size,
    seed=seed
)

mask_generator = mask_datagen.flow_from_directory(
    train_masks_dir,
    color_mode="grayscale",
    class_mode=None,
    batch_size = batch_size,
    seed=seed
)

# Combine the image and mask generators
train_generator = zip(image_generator, mask_generator)

num_train_imgs = len(os.listdir('blurred_images/train/train_images'))
steps_per_epoch = num_train_imgs //batch_size


Found 3 images belonging to 1 classes.
Found 3 images belonging to 1 classes.


In [37]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score
from tensorflow.keras.optimizers import Adam

# Load the pretrained model
BACKBONE = 'resnet50'

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=50)

checkpoint_path = "training_weights/resnet50/imcp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_save_weight = tf.keras.callbacks.ModelCheckpoint(checkpoint_dir,verbose=1,save_weights_only= True,period = 5)


# Set the input image dimensions
input_shape = (1024, 1024, 3)

model = sm.Unet(BACKBONE,encoder_weights='imagenet', input_shape=input_shape,classes=1)


# Compile the model with sparse categorical cross-entropy loss
# Define the optimizer and compile the model with the loss and metrics
optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss=bce_jaccard_loss, metrics=[iou_score,tf.keras.metrics.Precision()])

weights = tf.train.latest_checkpoint('training_weights/')
if  weights:
    model.load_weights(weights)
    

# Train the model
history = model.fit_generator(train_generator,epochs=400,
                    steps_per_epoch=steps_per_epoch,callbacks=[early_stopping_cb,cp_save_weight])

/var/folders/3y/1js5b25s1b1dk8g7drn7_g8h0000gn/T/ipykernel_16143/2951612275.py:34: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,epochs=400,


Epoch 1/400


2023-06-26 19:03:03.121038: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - ETA: 0s - loss: 1.1033 - iou_score: 5.2183e-10 - precision: 0.0000e+00WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,iou_score,precision


1/1 [==============================] - 4s 4s/step - loss: 1.1033 - iou_score: 5.2183e-10 - precision: 0.0000e+00
Epoch 2/400
1/1 [==============================] - ETA: 0s - loss: 1.0988 - iou_score: 5.4420e-10 - precision: 0.0000e+00WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,iou_score,precision


...


Epoch 40: saving model to training_weights/resnet50
1/1 [==============================] - 2s 2s/step - loss: 1.0304 - iou_score: 1.7019e-09 - precision: 0.0000e+00
Epoch 41/400
1/1 [==============================] - ETA: 0s - loss: 1.0297 - iou_score: 1.7387e-09 - precision: 0.0000e+00WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,iou_score,precision


1/1 [==============================] - 0s 387ms/step - loss: 1.0297 - iou_score: 1.7387e-09 - precision: 0.0000e+00
Epoch 42/400
1/1 [==============================] - ETA: 0s - loss: 1.0291 - iou_score: 1.7759e-09 - precision: 0.0000e+00WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,iou_score,precision


1/1 [==============================] - 0s 234ms/step - loss: 1.0291 - iou_score: 1.7759e-09 - precision: 0.0000e+00
Epoch 43/400
1/1 [==============================] - ETA: 0s - loss: 1.0285 - iou_score: 1.8132e-09 - precision: 0.0000e+00WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,iou_score,precision


1/1 [==============================] - 0s 235ms/step - loss: 1.0285 - iou_score: 1.8132e-09 - precision: 0.0000e+00
Epoch 44/400


KeyboardInterrupt: 